In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

#pune.ws

In [2]:
from urllib.parse import urlparse

In [3]:
url='https://www.pune.ws/in/companies/2/'

In [4]:
headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
webpage=requests.get(url,headers=headers)
print(webpage.status_code)

webpage=webpage.text

200


In [5]:
soup=BeautifulSoup(webpage,'html.parser')

In [6]:
company_div=soup.find('div',class_='ebl')
address_div = company_div.find('div', style='margin-top:0.5em')
address = address_div.text.strip() if address_div else 'N/A'
address

'Address : S.P. Infocity, Survey No - 209, Pune-Saswad Road, Phursungi, Hadapsar, Pune - 412308'

In [7]:
def extract_data(num_pages):
  data={'company_name':[],'address':[],'company_url':[]}

  for i in range(num_pages):
    url=f'https://www.pune.ws/in/companies/{i+1}/'
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    webpage=requests.get(url,headers=headers).text

    soup=BeautifulSoup(webpage,'html.parser')

    company_divs=soup.find_all('div',class_='ebl')
    for company_div in company_divs:
      # Extract the company name
      name_tag = company_div.find('h2', class_='etl')
      company_name = name_tag.text.strip() if name_tag else 'N/A'

      # Extract the company URL
      url_span = company_div.find('span', class_='ecl')
      company_url = url_span.find('a')['href'] if url_span and url_span.find('a') else 'N/A'

      address_div = company_div.find('div', style='margin-top:0.5em')
      address = address_div.text.strip() if address_div else 'N/A'

      # Print the extracted information
      if company_name and (company_url != 'N/A'):
        data['company_name'].append(company_name)
        data['address'].append(address)
        data['company_url'].append(company_url)

  return pd.DataFrame(data)


def extract_domain(url):
    parsed_uri = urlparse(url)
    domain = '{uri.netloc}'.format(uri=parsed_uri)
    if domain.startswith('www.'):
        domain = domain[4:]
    return domain


In [8]:
df=extract_data(15)
df['domain'] = df['company_url'].apply(extract_domain)
df = df[~df['domain'].str.contains('www-')]

In [9]:
df['career'] = 'careers@' + df['domain']
df['hr'] = 'hr@' + df['domain']
df['hiring'] = 'hiring@' + df['domain']

In [10]:
# rows_without_http = df[df['domain'].str.contains('www')]
# rows_without_http.head()

In [11]:
df.head()

,company_name,address,company_url,domain,career,hr,hiring
0,EXL Service Pune India,"Address : Tower 1, Ground Floor / First Floor,...",https://www.exlservice.com/,exlservice.com,careers@exlservice.com,hr@exlservice.com,hiring@exlservice.com
1,"Tata Consultancy Services / TCS, Pune",Address : Tata Research Development & Design C...,http://www.tcs.com,tcs.com,careers@tcs.com,hr@tcs.com,hiring@tcs.com
2,Accenture Services Pvt Ltd. Pune India,"Address : Building B-1, Magarpatta City (SEZ),...",https://www.accenture.com/in-en,accenture.com,careers@accenture.com,hr@accenture.com,hiring@accenture.com
3,"WNS Global Services, Pune India","Address : 3rd to 7th Floor, Wing A & B, Tower ...",http://www.wns.com,wns.com,careers@wns.com,hr@wns.com,hiring@wns.com
4,"Infosys Technologies, Pune India","Address : Plot No. 1, Rajiv Gandhi Infotech Pa...",http://www.infosys.com,infosys.com,careers@infosys.com,hr@infosys.com,hiring@infosys.com


In [12]:
df[~df['address'].str.contains('Address')].head()

,company_name,address,company_url,domain,career,hr,hiring
29,MphasiS,N/A,http://www.mphasis.com/ContactUs/Contact_ourOf...,mphasis.com,careers@mphasis.com,hr@mphasis.com,hiring@mphasis.com
35,"JoyTree Software, Pune India",N/A,http://joytree.in,joytree.in,careers@joytree.in,hr@joytree.in,hiring@joytree.in
49,PTC Software (India) Pvt. Ltd. / Parametric Te...,N/A,http://www.ptc.com,ptc.com,careers@ptc.com,hr@ptc.com,hiring@ptc.com
59,"RevoSolution, Pune India",N/A,http://www.revosolution.com,revosolution.com,careers@revosolution.com,hr@revosolution.com,hiring@revosolution.com
96,Pesh-Infotech Pvt. Ltd. Pune,N/A,http://peshinfotech.com,peshinfotech.com,careers@peshinfotech.com,hr@peshinfotech.com,hiring@peshinfotech.com
